In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import networkx as nx
import random
import numpy


def mc_upper_bound(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "chromatic_number" integer upper bound on the maximum clique number
	"""
	answ = nx.algorithms.coloring.greedy_color(G)
	chromatic_number = list(set(list(answ.values())))
	return len(chromatic_number)

def mc_lower_bound(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "lower bound" list of variables which form a clique in G
	"""
	return nx.maximal_independent_set(nx.complement(G))

def edge_k_core(G, k):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	 - "k" Integer that is at least one less than the global maximum clique number
	OUTPUT:
	 - "G" Networkx Undirected Graph where edge k-core reduction has been applied
	"""
	for a in list(G.edges()):
		x = list(G.neighbors(a[0]))
		y = list(G.neighbors(a[1]))
		if len(list(set(x) & set(y))) <= (k-2):
			G.remove_edge(a[0], a[1])
	return G

def k_core_reduction(graph, k):
	"""
	INPUT:
	 - "graph" Networkx Undirected Graph
	 - "k" Integer that is at least one less than the global maximum clique number
	OUTPUT:
	 - "graph" Networkx Undirected Graph where k-core reduction has been applied
	"""
	graph = nx.k_core(graph, k)
	ref1 = len(list(graph.edges()))
	graph = edge_k_core(graph, k)
	ref2 = len(list(graph.edges()))
	while ref1 != ref2:
		if len(graph) == 0:
			return graph
		graph = nx.k_core(graph, k)
		ref1 = len(list(graph.edges()))
		graph = edge_k_core(graph, k)
		ref2 = len(list(graph.edges()))
	return graph

def is_clique(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "True" if G is a clique, and "False" if G is not a clique
	"""
	n = len(list(G.nodes()))
	m = len(list(G.edges()))
	if int(m) == int((n*(n-1))/float(2)):
		return True
	else:
		return False

def ch_partitioning(vertex, G):
	"""
	INPUT:
	 - "vertex" splitting vertex
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "SSG" Left subgraph after partitioning
	 - "SG" Right subgraph after partitioning
	"""
	n = list(G.neighbors(vertex))
	Gp = []
	for iter in list(G.edges()):
		if iter[0] in n:
			if iter[1] in n:
				Gp.append(iter)
	G.remove_node(vertex)
	return nx.Graph(Gp), G

def lowest_degree_vertex(graph):
	"""
	INPUT:
	 - "graph" Networkx Undirected Graph
	OUTPUT:
	 - "i" node that has the lowest degree in the graph
	"""
	degrees = [graph.degree(a) for a in list(graph.nodes())]
	minimum = min(degrees)
	for i in list(graph.nodes()):
		if graph.degree(i) == minimum:
			return i

def remove_zero_degree_nodes(graph):
	"""
	INPUT:
	 - "graph" Networkx Undirected Graph
	OUTPUT:
	 - "graph" Networkx Undirected Graph with no zero degree nodes
	"""
	nodes = list(graph.nodes())
	for n in nodes:
		if graph.degree(n) == 0:
			graph.remove_node(n)
	return graph

def DBK(graph, LIMIT):
  """
  INPUT:
    - "graph" must be a Networkx Undirected Graph
    - "LIMIT" is an integer describing the largest size of graph which solver_func can solve; all subgraph sizes solved will be less than or equal to LIMIT
    - "solver_function" takes a Networkx Graph, and outputs a list of nodes which are hopefully the Maximum Clique elements; it can be an approximate or exact solver function
  OUTPUT:
    - "k" is a list of graph nodes which form a clique in the input graph. If the solver is exact, then k is the Maximum Clique
  NOTES:
    - The central idea of using bounds is that we maintain a global lower bound on the Maximum Clique. Then, for each sub problem we calculate a fast upper bound.
      If any sub problem has an upper bound which is less than or equal to the global lower bound, we can remove that sub problem from consideration in the remaining iterations of the algorithm.
    - This algorithm does not necessarily enumerate all cliques nor all Maximum Cliques. In particular, it is designed to return a single maximum clique assuming the solver is exact.
      However, the algorithm could be modified to include all maximum cliques found from solving each sub-problem.
    - There are many assert statements in this function. These all serve as "sanity checks"; if any of them are tripped, something went wrong or an input was incorrect
  """
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting DBK Algorithm ===")
  G = graph.copy()
  num_of_atoms = 0
  iteration = 0
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    num_of_atoms +=1
    print("=== Finished DBK Algorithm ===")
    return num_of_atoms, iteration, atom_sizes
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  graph = k_core_reduction(graph, len(k))
  if len(graph) == 0:
    return num_of_atoms, iteration, atom_sizes
  if len(graph) <= LIMIT:
    print("=== After K-core Reduction the Graph Size is Smaller than LIMIT ===")
    num_of_atoms += 1
    print("=== Finished DBK Algorithm ===")
    return num_of_atoms, iteration, atom_sizes
  vertex_removal = {graph: []}
  subgraphs = [graph]
  while len(subgraphs) != 0:
    iteration += 1
    SG = subgraphs.pop()
    SG = remove_zero_degree_nodes(SG)
    #print("Current subgraph size:", len(SG))
    assert len(SG) != 0
    vcount = vertex_removal[SG]
    del vertex_removal[SG]
    vertex = lowest_degree_vertex(SG)
    SSG, SG = ch_partitioning(vertex, SG)
    SG = remove_zero_degree_nodes(SG) # BIG
    SSG = remove_zero_degree_nodes(SSG) # SMALL
    SG = k_core_reduction(SG, len(k)-len(vcount)) # 0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) # 1
    vertex_removal[SSG] = vcount+[vertex]
    vertex_removal[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removal[SSG])) == True
      if len(SSG)+len(vertex_removal[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removal[SSG]
      del vertex_removal[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removal[SG])) == True
      if len(SG)+len(vertex_removal[SG]) > len(k):
        k = list(SG.nodes())+vertex_removal[SG]
      del vertex_removal[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removal[SSG]
      assert is_clique(G.subgraph(SSG_lower)) == True
      #print("SSG lower", len(SSG_lower))
      #print("lowerbound:", len(k))
      if len(SSG_lower) > len(k):
        vcount = vertex_removal[SSG]
        del vertex_removal[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removal[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removal[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SSG))
            atom_sizes.append(len(SSG))
            num_of_atoms += 1
            del vertex_removal[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removal[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removal.keys()):
        sub_solution_SSG = vertex_removal[SSG]
        del vertex_removal[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removal[SG]
      assert is_clique(G.subgraph(SG_lower)) == True
      #print("SG lower", len(SG_lower))
      #print("lowerbound:", len(k))
      if len(SG_lower) > len(k):
        vcount = vertex_removal[SG]
        del vertex_removal[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removal[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removal[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SG))
            atom_sizes.append(len(SG))
            num_of_atoms += 1
            # sub_solution_SG = solver_function(SG)+vertex_removal[SG]
            del vertex_removal[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removal[SG]
    if len(SG) == 0:
      if SG in list(vertex_removal.keys()):
        sub_solution_SG = vertex_removal[SG]
        del vertex_removal[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
  assert len(vertex_removal) == 0
  print("=== Finished DBK Algorithm ===")
  return num_of_atoms, iteration, atom_sizes

In [ ]:
from networkx.algorithms.cuts import node_expansion
from tqdm import tqdm
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

def ch_partitioning_eval(G, LIMIT, eval_function):
  k = mc_lower_bound(G)
  og_bound = len(k)
  graph = k_core_reduction(G, len(k))
  og_size = len(G)
  eval_list = []
  node_eval_list = []
  best_eval = -10000
  best_node = 0
  for vertex in tqdm(list(G.nodes())):
    graph = G.copy()
    vertex_removals = {graph: []}
    SG = graph
    subgraphs = [graph]
    vcount = vertex_removals[SG]
    del vertex_removals[SG]
    SSG, SG = ch_partitioning(vertex, graph)
    SG = remove_zero_degree_nodes(SG)
    SSG = remove_zero_degree_nodes(SSG)
    SG = k_core_reduction(SG, len(k)-len(vcount)) #0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) #1
    vertex_removals[SSG] = vcount+[vertex]
    vertex_removals[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removals[SSG])) == True
      if len(SSG)+len(vertex_removals[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removals[SSG]
      del vertex_removals[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removals[SG])) == True
      if len(SG)+len(vertex_removals[SG]) > len(k):
        k = list(SG.nodes())+vertex_removals[SG]
      del vertex_removals[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removals[SSG]
      #print(vertex_removals[SSG])
      assert is_clique(G.subgraph(SSG_lower)) == True
      if len(SSG_lower) > len(k):
        vcount = vertex_removals[SSG]
        del vertex_removals[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removals[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removals[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            del vertex_removals[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removals[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removals.keys()):
        sub_solution_SSG = vertex_removals[SSG]
        del vertex_removals[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removals[SG]
      #print(vertex_removals[SG])
      assert is_clique(G.subgraph(SG_lower)) == True
      if len(SG_lower) > len(k):
        vcount = vertex_removals[SG]
        del vertex_removals[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removals[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removals[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            del vertex_removals[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removals[SG]
    if len(SG) == 0:
      if SG in list(vertex_removals.keys()):
        sub_solution_SG = vertex_removals[SG]
        del vertex_removals[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
    # define eval parameters
    SG_size =  len(SG)
    if SG_size <= LIMIT:
      SG_size = 0
    SSG_size = len(SSG)
    if SSG_size <= LIMIT:
      SSG_size = 0
    bound_improvement = len(k) - og_bound
    current_eval = eval_function(og_size, SG_size, SSG_size, bound_improvement)
    eval_list.append(current_eval)
    node_eval_list.append(vertex)
    if current_eval > best_eval:
      best_eval = current_eval
      best_node = vertex
  temp = [[x, y] for x, y in zip(node_eval_list, eval_list)]
  node_eval_list = temp
  return node_eval_list, best_eval, best_node

In [ ]:
# DBK2 with the new eval functions

def DBK2(graph, LIMIT, eval_function):
  """
  INPUT:
    - "graph" must be a Networkx Undirected Graph
    - "LIMIT" is an integer describing the largest size of graph which solver_func can solve; all subgraph sizes solved will be less than or equal to LIMIT
    - "solver_function" takes a Networkx Graph, and outputs a list of nodes which are hopefully the Maximum Clique elements; it can be an approximate or exact solver function
  OUTPUT:
    - "k" is a list of graph nodes which form a clique in the input graph. If the solver is exact, then k is the Maximum Clique
  NOTES:
    - The central idea of using bounds is that we maintain a global lower bound on the Maximum Clique. Then, for each sub problem we calculate a fast upper bound.
      If any sub problem has an upper bound which is less than or equal to the global lower bound, we can remove that sub problem from consideration in the remaining iterations of the algorithm.
    - This algorithm does not necessarily enumerate all cliques nor all Maximum Cliques. In particular, it is designed to return a single maximum clique assuming the solver is exact.
      However, the algorithm could be modified to include all maximum cliques found from solving each sub-problem.
    - There are many assert statements in this function. These all serve as "sanity checks"; if any of them are tripped, something went wrong or an input was incorrect
  """
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting DBK Algorithm ===")
  G = graph.copy()
  num_of_atoms = 0
  iteration = 0
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    num_of_atoms +=1
    print("=== Finished DBK Algorithm ===")
    return num_of_atoms, iteration, atom_sizes
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  graph = k_core_reduction(graph, len(k))
  if len(graph) == 0:
    return k
  if len(graph) <= LIMIT:
    print("=== After K-core Reduction the Graph Size is Smaller than LIMIT ===")
    num_of_atoms += 1
    print("=== Finished DBK Algorithm ===")
    return num_of_atoms, iteration, atom_sizes
  vertex_removal = {graph: []}
  subgraphs = [graph]
  while len(subgraphs) != 0:
    iteration += 1
    #print("Iteration number:", iteration)
    SG = subgraphs.pop()
    SG = remove_zero_degree_nodes(SG)
    #print("Current subgraph size:", len(SG))
    assert len(SG) != 0
    vcount = vertex_removal[SG]
    del vertex_removal[SG]
    #print("LOOKING FOR BEST VERTEX")
    SG_copy = SG.copy()
    _,_,vertex = ch_partitioning_eval(SG_copy, LIMIT, average_node_decrease)
    SSG, SG = ch_partitioning(vertex, SG)
    SG = remove_zero_degree_nodes(SG) # BIG
    SSG = remove_zero_degree_nodes(SSG) # SMALL
    SG = k_core_reduction(SG, len(k)-len(vcount)) # 0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) # 1
    vertex_removal[SSG] = vcount+[vertex]
    vertex_removal[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removal[SSG])) == True
      if len(SSG)+len(vertex_removal[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removal[SSG]
      del vertex_removal[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removal[SG])) == True
      if len(SG)+len(vertex_removal[SG]) > len(k):
        k = list(SG.nodes())+vertex_removal[SG]
      del vertex_removal[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removal[SSG]
      assert is_clique(G.subgraph(SSG_lower)) == True
      if len(SSG_lower) > len(k):
        vcount = vertex_removal[SSG]
        del vertex_removal[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removal[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removal[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SSG))
            atom_sizes.append(len(SSG))
            num_of_atoms += 1
            del vertex_removal[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removal[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removal.keys()):
        sub_solution_SSG = vertex_removal[SSG]
        del vertex_removal[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removal[SG]
      assert is_clique(G.subgraph(SG_lower)) == True
      if len(SG_lower) > len(k):
        vcount = vertex_removal[SG]
        del vertex_removal[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removal[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removal[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SG))
            atom_sizes.append(len(SG))
            num_of_atoms += 1
            # sub_solution_SG = solver_function(SG)+vertex_removal[SG]
            del vertex_removal[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removal[SG]
    if len(SG) == 0:
      if SG in list(vertex_removal.keys()):
        sub_solution_SG = vertex_removal[SG]
        del vertex_removal[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
  assert len(vertex_removal) == 0
  print("=== Finished DBK Algorithm ===")
  return num_of_atoms, iteration, atom_sizes

In [ ]:
!pip install torch_geometric

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from torch_geometric.data import Data
import numpy as np

# Defining some data preprocessing functions
def graph_to_pyg_data(graph):
    edge_index = torch.tensor(list(graph.edges)).t().contiguous()
    x = torch.eye(graph.number_of_nodes())  # Node features, identity matrix in this example
    y = torch.tensor(list(nx.get_node_attributes(graph, 'label').values()))  # Node labels

    return Data(x=x, edge_index=edge_index, y=y)

def to_networkx(data):
    edge_index = data.edge_index.cpu().numpy()
    edge_attr = None
    if data.edge_attr is not None:
        edge_attr = data.edge_attr.cpu().numpy()

    G = nx.Graph()
    G.add_nodes_from(range(data.num_nodes))
    G.add_edges_from(edge_index.T)

    if edge_attr is not None:
        for i, (src, tgt) in enumerate(edge_index.T):
            G[src][tgt]['edge_attr'] = edge_attr[i]

    return G

    # min max normalize [0, 1]
def min_max_normalize(data, new_min=0, new_max=1):
    # Find the min and max values in the data
    min_val = min(data)
    max_val = max(data)
    if min_val == max_val:
      return [1 for _ in range(len(data))]
    normalized_data = [(x - min_val) / (max_val - min_val) * (new_max - new_min) + new_min for x in data]
    return normalized_data

# top k% of values are 1 rest are 0
def top_k(label_list, k):
    k = round(k*len(label_list))
    input_list = min_max_normalize(label_list)
    # Sort the list in descending order
    sorted_list = sorted(input_list, reverse=True)
    # Determine the threshold index
    threshold_index = min(k, len(sorted_list))
    # Set the first k elements to 1 and the rest to 0
    thresholded_list = [1 if i < threshold_index else 0 for i in range(len(sorted_list))]
    # Create a mapping from original indices to sorted indices
    index_mapping = {original: sorted_index for sorted_index, original in enumerate(sorted(range(len(input_list)), key=lambda x: input_list[x], reverse=True))}
    # Sort the thresholded list back to the original order
    thresholded_list_original_order = [thresholded_list[index_mapping[i]] for i in range(len(input_list))]

    return thresholded_list_original_order

# continuous [0,1]
def continuous(label_list, k):
  label_list = min_max_normalize(label_list)
  return label_list

# value above k are 1 rest are 0
def within_k(label_list, k):
  k = 1-k
  label_list = min_max_normalize(label_list)
  data = [1 if x > k else 0 for x in label_list]
  return data

In [ ]:
# SOME EVALUATION FUNCTIONS

# average of |G| - |SG| and |G| - |SSG|
def average_node_decrease(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  if subgraph1_size == 0 and subgraph2_size == 0:
    return original_graph_size
  elif subgraph1_size == 0:
    return original_graph_size - subgraph2_size
  elif subgraph2_size == 0:
    return original_graph_size - subgraph1_size
  else:
    return (2*original_graph_size - subgraph2_size - subgraph1_size)/2

# average of |G| - |SG| and |G| - |SSG| PLUS bound improvement
def average_node_decrease_with_bound(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  if subgraph1_size == 0 and subgraph2_size == 0:
    return original_graph_size + bound_improve
  elif subgraph1_size == 0:
    return original_graph_size - subgraph2_size + bound_improve
  elif subgraph2_size == 0:
    return original_graph_size - subgraph1_size + bound_improve
  else:
    return (2*original_graph_size - subgraph2_size - subgraph1_size)/2 + bound_improve

# total node decrease
def total_node_decrease(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  return 2*original_graph_size - subgraph1_size - subgraph2_size

# total node decrease PLUS bound improvement
def total_node_decrease(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  return 2*original_graph_size - subgraph1_size - subgraph2_size + bound_improve

# max node decrease
def max_node_decrease(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  return original_graph_size - max(subgraph1_size, subgraph2_size)

# bound improvement
def total_bound_improvement(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  return bound_improve



In [ ]:
# Generating Labels Function
def label_generation(graph, LIMIT, eval_function=average_node_decrease):
  """
  Input: networkx graph, decomposition LIMIT
  - Preprocess graph (kcore)
  - Assign labels to each node
  Output: networkx graph with node labels
  """
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting Labelling Algorithm ===")
  G = graph.copy()
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    return
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  vertex_removal = {graph: []}
  subgraphs = [graph]
  SG = subgraphs.pop()
  SG = remove_zero_degree_nodes(SG)
  assert len(SG) != 0
  vcount = vertex_removal[SG]
  del vertex_removal[SG]
  print("Computing Labels")
  SG_copy = SG.copy()
  eval_list,_,_ = ch_partitioning_eval(SG_copy, LIMIT, eval_function)
  for node_eval in eval_list:
    node = node_eval[0]
    eval = node_eval[1]
    G.nodes[node]['label'] = eval
  return G

# Normalize dataset function
def dataset_normalize(dataset, normalize_function, normalize_param):
  for graph_idx in tqdm(range(len(dataset))):
    data = dataset[graph_idx]
    node_labels_list = data.y.tolist()
    normalized_node_labels_list = normalize_function(node_labels_list, normalize_param)
    data.y = torch.tensor(normalized_node_labels_list)
  return dataset

In [ ]:
# TAKES IN A NETWORKX GRAPH AND OUTPUTS A TENSOR THAT IS THE NODE FEATURES
def get_features(nxgraph):
  # get features
  node_degrees = dict(nxgraph.degree())
  degree_centrality = nx.degree_centrality(nxgraph)
  betweenness_centrality = nx.betweenness_centrality(nxgraph)
  closeness_centrality = nx.closeness_centrality(nxgraph)
  eigenvector_centrality = nx.eigenvector_centrality(nxgraph)
  pagerank_centrality = nx.pagerank(nxgraph)
  harmonic_centrality = nx.harmonic_centrality(nxgraph)
  load_centrality = nx.load_centrality(nxgraph)
  clustering_coefficient = nx.clustering(nxgraph)
  # make it into an array
  features_array = np.array([
    list(node_degrees.values()),
    list(degree_centrality.values()),
    list(betweenness_centrality.values()),
    list(closeness_centrality.values()),
    list(eigenvector_centrality.values()),
    list(pagerank_centrality.values()),
    list(harmonic_centrality.values()),
    list(load_centrality.values()),
    list(clustering_coefficient.values())])
  features_array = features_array.T
  return torch.tensor(features_array)


In [ ]:
# TRAINING DATASET GENERATION
LIMIT = 40
labeled_dataset = []
eval_function = average_node_decrease
number_of_graphs = 500
print("Preprocessing and Generating Labels...")
for i in tqdm(range(number_of_graphs)):
  G = nx.gnp_random_graph(200, random.uniform(0.3, 0.9))
  labeled_graph = label_generation(G, LIMIT=LIMIT, eval_function=eval_function)
  if labeled_graph is None:
    continue
  #feature_tensor = get_features(labeled_graph)
  final_graph = graph_to_pyg_data(labeled_graph)
  #final_graph.x = feature_tensor
  labeled_dataset.append(final_graph)
  print(final_graph)

torch.save(labeled_dataset, "/content/datasets/train_200nodes_500graphs.pt")

In [ ]:
# TEST SET 200 nodes different densities
LIMIT = 40
labeled_dataset = []
eval_function = average_node_decrease
number_of_graphs = 10
print("Preprocessing and Generating Labels...")
for i in tqdm(range(number_of_graphs)):
  G = nx.gnp_random_graph(80, random.uniform(0.1, 0.7))
  labeled_graph = label_generation(G, LIMIT=LIMIT, eval_function=eval_function)
  if labeled_graph is None:
    continue
  #feature_tensor = get_features(labeled_graph)
  final_graph = graph_to_pyg_data(labeled_graph)
  #final_graph.x = feature_tensor
  labeled_dataset.append(final_graph)
  print(final_graph)

torch.save(labeled_dataset, "/content/test_80nodes_10graphs.pt")

Preprocessing and Generating Labels...


  0%|          | 0/10 [00:00<?, ?it/s]

=== Starting Labelling Algorithm ===
Preprocessing...
Computing Labels


 10%|█         | 1/10 [00:03<00:29,  3.31s/it]

Data(x=[80, 80], edge_index=[2, 2045], y=[80])
=== Starting Labelling Algorithm ===
Preprocessing...
Computing Labels


...

Data(x=[80, 80], edge_index=[2, 987], y=[80])
=== Starting Labelling Algorithm ===
Preprocessing...
Computing Labels


 90%|█████████ | 9/10 [00:20<00:02,  2.23s/it]

Data(x=[80, 80], edge_index=[2, 1579], y=[80])
=== Starting Labelling Algorithm ===
Preprocessing...
Computing Labels


100%|██████████| 10/10 [00:22<00:00,  2.26s/it]

Data(x=[80, 80], edge_index=[2, 989], y=[80])


In [ ]:
# VAL SET 200 nodes different densities
LIMIT = 40
labeled_dataset = []
eval_function = average_node_decrease
number_of_graphs = 200
print("Preprocessing and Generating Labels...")
for i in tqdm(range(number_of_graphs)):
  G = nx.gnp_random_graph(200, random.uniform(0.1, 0.7))
  labeled_graph = label_generation(G, LIMIT=LIMIT, eval_function=eval_function)
  if labeled_graph is None:
    continue
  #feature_tensor = get_features(labeled_graph)
  final_graph = graph_to_pyg_data(labeled_graph)
  #final_graph.x = feature_tensor
  labeled_dataset.append(final_graph)
  print(final_graph)

torch.save(labeled_dataset, "/content/datasets/val_200nodes_200graphs.pt")

In [ ]:
# TEST SET 300/400/500 nodes 0.5 density
LIMIT = 40
labeled_dataset = []
eval_function = average_node_decrease
number_of_graphs = 10
print("Preprocessing and Generating Labels...")
for i in tqdm(range(number_of_graphs)):
  G = nx.gnp_random_graph(300, 0.5)
  labeled_graph = label_generation(G, LIMIT=LIMIT, eval_function=eval_function)
  if labeled_graph is None:
    continue
  #feature_tensor = get_features(labeled_graph)
  final_graph = graph_to_pyg_data(labeled_graph)
  #final_graph.x = feature_tensor
  labeled_dataset.append(final_graph)
  print(final_graph)

torch.save(labeled_dataset, "/content/datasets/test_300nodes_10graphs.pt")

labeled_dataset = []
for i in tqdm(range(number_of_graphs)):
  G = nx.gnp_random_graph(400, 0.5)
  labeled_graph = label_generation(G, LIMIT=LIMIT, eval_function=eval_function)
  if labeled_graph is None:
    continue
  #feature_tensor = get_features(labeled_graph)
  final_graph = graph_to_pyg_data(labeled_graph)
  #final_graph.x = feature_tensor
  labeled_dataset.append(final_graph)
  print(final_graph)

torch.save(labeled_dataset, "/content/datasets/test_400nodes_10graphs.pt")

labeled_dataset = []
for i in tqdm(range(number_of_graphs)):
  G = nx.gnp_random_graph(500, 0.5)
  labeled_graph = label_generation(G, LIMIT=LIMIT, eval_function=eval_function)
  if labeled_graph is None:
    continue
  #feature_tensor = get_features(labeled_graph)
  final_graph = graph_to_pyg_data(labeled_graph)
  #final_graph.x = feature_tensor
  labeled_dataset.append(final_graph)
  print(final_graph)

torch.save(labeled_dataset, "/content/datasets/test_500nodes_10graphs.pt")